# 05. API Integration (External Data → Reasoning)

외부 API (예: Serpapi)를 호출해 **실시간 데이터**를 받아 모델이 **컨텍스트로 활용**하도록 합니다.

## 준비
- `SERP_API_KEY` 환경 변수를 설정하세요.
- 도시명을 입력하면 현재 날씨를 조회 후 한 줄 요약을 생성합니다.


In [28]:
from dotenv import load_dotenv
load_dotenv()

import os, requests
from openai import AzureOpenAI

# Azure OpenAI 환경 변수 확인
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

assert AZURE_OPENAI_ENDPOINT, "AZURE_OPENAI_ENDPOINT가 필요합니다."
assert AZURE_OPENAI_KEY, "AZURE_OPENAI_KEY가 필요합니다."
assert SERP_API_KEY, "SERP_API_KEY가 필요합니다."

# Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version="2024-02-15-preview"
)

In [ ]:
import json
import requests

def get_weather(city: str) -> dict:
    """SerpAPI를 사용해서 Google 검색을 통해 날씨 정보 가져오기"""
    
    try:
        # SerpAPI 직접 HTTP 요청
        params = {
            "q": f"{city} 날씨",
            "api_key": os.getenv("SERP_API_KEY"),
            "location": "South Korea",
            "hl": "ko",
            "gl": "kr",
            "engine": "google"
        }
        
        response = requests.get("https://serpapi.com/search", params=params)
        response.raise_for_status()
        results = response.json()
        
        # 날씨 정보 추출
        weather_info = {}
        
        # answer_box에서 날씨 정보 추출
        if "answer_box" in results and "type" in results["answer_box"]:
            answer_box = results["answer_box"]
            
            if answer_box["type"] == "weather_result":
                # 실제 Google 날씨 데이터 사용
                weather_info = {
                    "city": city,
                    "temperature": f"{answer_box.get('temperature', '정보 없음')}°C",
                    "description": answer_box.get('weather', answer_box.get('precipitation', '맑음')),
                    "humidity": answer_box.get('humidity', '정보 없음'),
                    "wind": answer_box.get('wind', '정보 없음'),
                    "location": answer_box.get('location', city),
                    "date": answer_box.get('date', '현재'),
                    "precipitation": answer_box.get('precipitation', '0%')
                }
                print(f"✅ 실제 날씨 데이터 추출 성공: {weather_info['location']} {weather_info['temperature']}")
        
        # 날씨 정보를 찾지 못한 경우 기본값
        if not weather_info:
            print("⚠️ 날씨 정보를 찾지 못해 기본값 사용")
            weather_info = {
                "city": city,
                "temperature": "22°C",
                "description": "맑음",
                "source": "기본값",
                "snippet": f"{city}의 날씨 정보를 가져올 수 없어 기본값을 사용합니다."
            }
        
        return weather_info
    
    except Exception as e:
        print(f"SerpAPI 호출 중 오류 발생: {e}")
        # 오류 발생 시 기본값 반환
        weather_info = {
            "city": city,
            "temperature": "22°C",
            "description": "맑음",
            "source": "기본값 (오류로 인한)",
            "snippet": f"API 호출 중 오류가 발생했습니다: {str(e)}"
        }
        return weather_info

In [30]:
def summarize_weather(city: str, weather_data: dict, model=None) -> str:
    """SerpAPI로 가져온 날씨 데이터를 기반으로 요약 생성"""
    
    # Azure OpenAI 배포명 사용
    if model is None:
        model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    
    # 타입 확인 및 안전한 데이터 추출
    if not isinstance(weather_data, dict):
        print(f"경고: weather_data가 dict가 아님. 타입: {type(weather_data)}")
        weather_data = {"city": city, "temperature": "22°C", "description": "맑음"}
    
    # 안전한 정보 추출
    temp = weather_data.get("temperature", "정보 없음")
    desc = weather_data.get("description", "날씨 정보 없음")
    humidity = weather_data.get("humidity", "")
    wind = weather_data.get("wind", "")
    source = weather_data.get("source", "")
    snippet = weather_data.get("snippet", "")
    
    # 추가 정보가 있으면 포함
    extra_info = []
    if humidity and humidity != "정보 없음":
        extra_info.append(f"습도 {humidity}")
    if wind and wind != "정보 없음":
        extra_info.append(f"바람 {wind}")
    
    extra_text = f", {', '.join(extra_info)}" if extra_info else ""
    
    # 프롬프트 구성
    if snippet and snippet != "날씨 정보를 찾을 수 없습니다.":
        msg = f"{city}의 현재 상황: {snippet} 기온은 {temp}입니다. 이를 참고해 1) 짧은 일기 한 문장 2) 추천 활동 1가지를 제안하세요."
    else:
        msg = f"{city} 현재 날씨는 '{desc}', 기온은 {temp}{extra_text}입니다. 이를 참고해 1) 짧은 일기 한 문장 2) 추천 활동 1가지를 제안하세요."
    
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "당신은 날씨 정보를 바탕으로 친근하고 유용한 조언을 해주는 AI입니다."},
                {"role": "user", "content": msg}
            ],
            max_tokens=200,
            temperature=0.7
        )
        return completion.choices[0].message.content.strip()
    
    except Exception as e:
        return f"AI 응답 생성 중 오류가 발생했습니다: {str(e)}"

## Try it

In [31]:
city = "Seoul"
w = get_weather(city)
summarize_weather(city, w)

SerpAPI 응답 키들: ['search_metadata', 'search_parameters', 'search_information', 'inline_images', 'answer_box', 'organic_results', 'related_searches', 'pagination', 'serpapi_pagination']
Answer box 내용: {'type': 'weather_result', 'temperature': '25', 'unit': 'Celsius', 'precipitation': '10%', 'humidity': '58%', 'wind': '2m/s', 'location': '서울특별시', 'date': '(목요일) 오후 2:00', 'weather': '흐림', 'forecast': [{'day': '목요일', 'temperature': {'high': '25', 'low': '20'}, 'thumbnail': 'https://serpapi.com/searches/68de107c2efef84a1f00c8f1/images/b838fa759d954c3f3264127169a5dedf11bd3a1a1f58f767d3fb464f5e7e4b8c.gif', 'weather': '흐리고 때때로 갬', 'humidity': '65%', 'precipitation': '10%', 'wind': '2m/s'}, {'day': '금요일', 'temperature': {'high': '24', 'low': '19'}, 'thumbnail': 'https://serpapi.com/searches/68de107c2efef84a1f00c8f1/images/b838fa759d954c3f3264127169a5dedfb89c396f7febf464b230e4867462b2f6.gif', 'weather': '흐림', 'humidity': '63%', 'precipitation': '10%', 'wind': '4m/s'}, {'day': '토요일', 'temperature'

'1) 오늘 서울은 기온이 22°C로 따뜻하고 화창한 날씨가 이어져, 기분 좋은 하루가 될 것 같아요.\n\n2) 이런 날씨에는 공원에서 산책하거나 피크닉을 즐기는 것이 좋겠어요. 친구나 가족과 함께 자연 속에서 여유를 만끽해 보세요!'